In [1]:
# 강아지를 비문으로 인식할 수 있는 다중 분류 딥러닝 모델
# 패키지 불러오기
import pandas as pd
import numpy as np
import random as rd
import cv2, os

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.backend import clear_session
from keras.optimizers import Adam
from keras.datasets import mnist, fashion_mnist
import glob

In [ ]:
# # json 파일 불러오기
# import os
#
# json_files = []
#
# for i in range(1, 25):
#     if i not in [21, 23]:
#         json_file = f"D:\\Dog\\noseprints\\data\\a{i:03d}.json"
#         json_files.append(json_file)

In [ ]:
# import json
# import os
#
# # JSON 파일 범위 지정
# json_range = list(range(1, 25))
# json_range.remove(21)  # a021.json 제외
# json_range.remove(23)  # a023.json 제외
#
# base_path = "D:/Dog/noseprints/data" # base_path를 현재 작업물이 있는 폴더로 지정
#
# for i in json_range:
#     input_file = os.path.join(base_path, f"a{i:03d}.json")
#
#     # JSON 파일 읽기
#     with open(input_file, 'r', encoding='utf-8') as file:
#         data = json.load(file)
#
#     # "file_name" 필드 수정
#     for image in data["images"]:
#         image_id = image["id"]
#         new_file_name = f"a{i:03d}_{image_id}.jpg"
#         image["file_name"] = new_file_name
#
#     # 변경된 내용을 파일에 다시 저장
#     output_file = input_file
#     with open(output_file, 'w', encoding='utf-8') as file:
#         json.dump(data, file, ensure_ascii=False, indent=4)

In [ ]:
# import json
# merged_data = []
#
# # 각 json 파일을 읽고 병합
# for json_file in json_files:
#     with open(json_file, 'r', encoding='utf-8') as file:
#         data = json.load(file)
#         merged_data.append(data)
#
# output_path = "D:\\Dog\\noseprints\\data"
# os.makedirs(output_path, exist_ok=True)
#
# # 병합된 데이터를 D:\Dog\noseprints\data\labels\label.json 파일에 저장
# output_file = os.path.join(output_path, "label.json")
# with open(output_file, 'w', encoding='utf-8') as outfile:
#     json.dump(merged_data, outfile, ensure_ascii=False, indent=4)

In [ ]:
# python example.py --datasets COCO --img_path ../noseprints/data/images/a024  --label ../noseprints/data/a024.json --convert_output_path ../noseprints/data/labels --img_type '.jpg' --manifest_path ./ --cls_list_file ../noseprints/class.names

In [ ]:
# # json 파일을 YOLO 포맷으로 변환
# import subprocess
#
# def run_command(img_path, label_path):
#     command = f"python D:/Dog/convert2Yolo/example.py --datasets COCO --img_path {img_path} --label {label_path} --convert_output_path D:/Dog/noseprints/data/labels --img_type '.jpg' --manifest_path D:/Dog --cls_list_file D:/Dog/noseprints/class.names"
#
#     result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, shell=True, text=True)
#
#     print(result.stdout)
#     print(result.stderr)
#
# # 각 경로에 대한 루프를 실행
# for i in range(1, 25):
#     if i in [21, 23]:
#         continue
#     img_path = f"D:/Dog/noseprints/data/images/a0{i}"
#     label_path = f"D:/Dog/noseprints/data/a0{i}.json"
#     run_command(img_path, label_path)


In [ ]:
# 이미지를 train 데이터셋과 test 데이터셋으로 구분
from sklearn.model_selection import train_test_split
X = np.array(glob.glob("D:/Dog/noseprints/data/images/*.jpg")) # 이미지 데이터
y= np.array(glob.glob("D:/Dog/noseprints/data/labels/*.txt")) # 라벨 데이터

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, stratify=y, random_state=42)
# stratify 사용 시 이미지 갯수가 너무 적으면 아예 분리가 안됨

In [ ]:
# 이미지 파일과 라벨 파일을 정렬하고, 파일 이름을 기준으로 데이터셋을 분할
import numpy as np
import glob
from sklearn.model_selection import train_test_split

def extract_filename(file_path):
    return os.path.splitext(os.path.basename(file_path))[0]

image_files = np.array(sorted(glob.glob("D:/Dog/noseprints/data/images/*.jpg")))
label_files = np.array(sorted(glob.glob("D:/Dog/noseprints/data/labels/*.txt")))

image_filenames = np.array([extract_filename(f) for f in image_files])
label_filenames = np.array([extract_filename(f) for f in label_files])

assert np.array_equal(image_filenames, label_filenames), "이미지와 라벨 파일이 일치하지 않음."

X_train, X_test, y_train, y_test = train_test_split(image_files, label_files, test_size=0.1, random_state=42)

In [ ]:
# train 데이터셋과 test 데이터셋을 각각 train, test 폴더에 복사
import shutil
import os

def copy_files_to_folder(file_list, destination_folder):
    for file in file_list:
        shutil.copy(file, destination_folder)

# Train 데이터셋과 Test 데이터셋을 각각 train, test 폴더에 복사
train_image_folder = 'D:/Dog/noseprints/data/train/images/'
train_label_folder = 'D:/Dog/noseprints/data/train/labels/'
test_image_folder = 'D:/Dog/noseprints/data/test/images/'
test_label_folder = 'D:/Dog/noseprints/data/test/labels/'

# 폴더가 없는 경우 생성
os.makedirs(train_image_folder, exist_ok=True)
os.makedirs(train_label_folder, exist_ok=True)
os.makedirs(test_image_folder, exist_ok=True)
os.makedirs(test_label_folder, exist_ok=True)

# 이미지 파일 복사
copy_files_to_folder(X_train, train_image_folder)
copy_files_to_folder(X_test, test_image_folder)

# 라벨 파일 복사
copy_files_to_folder(y_train, train_label_folder)
copy_files_to_folder(y_test, test_label_folder)

In [ ]:
# 이미지 resize 시 라벨링 데이터도 같이 수정해야 함..진행하지 않기로 결정

In [ ]:
# yaml 파일 경로 : D:/Dog/noseprints/dog.yaml
# test data의 경우, 라벨링을 정보를 활용하지 않음
import yaml

In [ ]:
# yaml 파일을 읽어서 데이터셋을 생성
with open("D:/Dog/noseprints/dog.yaml", 'r') as yaml_file:
    data = yaml.load(yaml_file, Loader=yaml.FullLoader)
    print(data)

# yolov5로 경로이동
cd ../yolov5

# train
'''
python train.py \
--img 이미지 사이즈 \
--batch 배치 사이즈 \
--data custom.yaml 파일경로 \
--cfg 사용할 모델의 yaml 파일경로 \
--weights 학습에 사용할 모델 \
--name 학습된 정보를 runs 폴더 안에 저장할 이름 \
--project wanbd에 저장할 프로젝트명
'''

# # 여러 파라미터 중 핵심만 사용
# python train.py \
# --data ../starbucks/custom.yaml \
# --weights yolov5m.pt
#
# # 조금 더 조절하고 싶은 경우
# python train.py \
# --data ../starbucks/custom.yaml \
# --weights yolov5m.pt \
# --batch ?? \
# --epochs ??? \
# --name starbucks

In [2]:
# clear_session
from tensorflow.keras import backend as K
K.clear_session()

In [ ]:
# 학습된 모델을 이용하여 train
!python train.py --batch 16 --epochs 50 --data D:/Dog/dog.yaml --weights yolov5m.pt --name dog1 --hyp hyp.yaml --cache

In [ ]:
hyp = {'lr0': 0.01,  # initial learning rate
       'lrf': 0.2,  # final learning rate
       'momentum': 0.937,  # SGD momentum
       'weight_decay': 0.0005,  # optimizer weight decay
       'warmup_epochs': 3.0,  # warmup epochs
       'warmup_momentum': 0.8,  # warmup initial momentum
       'warmup_bias_lr': 0.1,  # warmup initial bias lr
       'box': 0.05,  # box loss gain
       'cls': 0.5,  # cls loss gain
       'cls_pw': 1.0,  # cls BCELoss positive_weight
       'obj': 1.0,  # obj loss gain (scale with pixels)
       'obj_pw': 1.0,  # obj BCELoss positive_weight
       'iou_t': 0.20,  # IoU training threshold
       'anchor_t': 4.0,  # anchor-multiple threshold
       'fl_gamma': 0.0,  # focal loss gamma (efficientDet default gamma=1.5)
       'hsv_h': 0.015,  # image HSV-Hue augmentation (fraction)
       'hsv_s': 0.7,  # image HSV-Saturation augmentation (fraction)
       'hsv_v': 0.4,  # image HSV-Value augmentation (fraction)
       'degrees': 0.0,  # image rotation (+/- deg)
       'translate': 0.1,  # image translation (+/- fraction)
       'scale': 0.5,  # image scale (+/- gain)
       'shear': 0.0,  # image shear (+/- deg)
       'perspective': 0.0,  # image perspective (+/- fraction), range 0-0.001
       'flipud': 0.0,  # image flip up-down (probability)
       'fliplr': 0.5, # image flip left-right (probability)
       'mosaic': 1.0, # image mixup (probability)
       'mixup': 0.0 # image mixup (probability)
       'copy_paste': 0.0, # segment copy-paste (probability)
       'visual': 0.0, # visualize augmentation (probability)
       'multi_scale': 0.0, # multi-scale training (probability)
       'label_smoothing': 0.0, # label smoothing (probability)
       'enable_mixup': True, # enable mixup training
       'enable_mosaic': True, # enable mosaic training
       'enable_cutmix': False, # enable cutmix training
       'enable_copypaste': False, # enable cutmix training
       'input_size': 640, # input size
       'random_size': False, # use multi-size training
       'random_size_min': 14, # min input size (pixels)
       'random_size_max': 26, # max input size (pixels)
       'random_size_interval': 4, # interval between sizes (pixels)
       'random_size_mode': 'rect', # rect | ceil | floor
       'letter_box': False, # letterbox resize
       'mosaic_border': [-10, 10], # mosaic border (pixels)
       'mixup_scale': (0.5, 1.5), # mixup scale
       'mixup_ratio': 0.5, # mixup ratio
       'cutmix_ratio': 0.5, # cutmix ratio
       'cutmix_beta': 1.0, # cutmix beta
       'cutmix_prob': 1.0, # cutmix probability
       'cutmix_minmax': None, # cutmix min/max ratio
       'cutmix_pad': False, # apply cutmix padding
       'cutmix_pseudo': False, # use pseudo segmentation
       'cutmix_criterion': 'iou', # cutmix IoU loss type (iou | iou_loss | bce)
       'cutmix_threshold': 0.5, # cutmix IoU threshold
       'cutmix_mosaic': False, # apply cutmix mosaic
       'early_stop': True, # early stop training
       'early_stop_patience': 10, # early stop patience epochs
       'gradient_accumulation_steps': 1, # gradient accumulation steps
       'workers': 8, # dataloader threads
       'image_weights': False, # use weighted image selection for training
       'compute_loss': None, # compute loss (train, val, test, None)
       'save_optimizer': True, # save optimizer to file
       'save_optimizer_path': '', # optimizer file path
       'save_optimizer_type': 'torch', # optimizer file type (torch, state_dict)
       'save_optimizer_every_epoch': False, # save optimizer every epoch
       'save_optimizer_every_epoch_path': '', # optimizer every epoch file path
       'save_optimizer_every_epoch_type': 'torch', # optimizer every epoch file type (torch, state_dict)
       'save_optimizer_last': True, # save optimizer to last checkpoint
       'save_optimizer_last_path': '', # optimizer last file path
       'save_optimizer_last_type': 'torch', # optimizer last file type (torch, state_dict)
       }

In [ ]:
# lr0: 0.01  # initial learning rate (SGD=1E-2, Adam=1E-3)
# lrf: 0.01  # final OneCycleLR learning rate (lr0 * lrf)
# momentum: 0.937  # SGD momentum/Adam beta1
# weight_decay: 0.0005  # optimizer weight decay 5e-4
# warmup_epochs: 3.0  # warmup epochs (fractions ok)
# warmup_momentum: 0.8  # warmup initial momentum
# warmup_bias_lr: 0.1  # warmup initial bias lr
# box: 0.05  # box loss gain
# cls: 0.5  # cls loss gain
# cls_pw: 1.0  # cls BCELoss positive_weight
# obj: 1.0  # obj loss gain (scale with pixels)
# obj_pw: 1.0  # obj BCELoss positive_weight
# iou_t: 0.20  # IoU training threshold
# anchor_t: 4.0  # anchor-multiple threshold
# # anchors: 3  # anchors per output layer (0 to ignore)
# fl_gamma: 0.0  # focal loss gamma (efficientDet default gamma=1.5)
# hsv_h: 0.015  # image HSV-Hue augmentation (fraction)
# hsv_s: 0.7  # image HSV-Saturation augmentation (fraction)
# hsv_v: 0.4  # image HSV-Value augmentation (fraction)
# degrees: 0.5  # image rotation (+/- deg)
# translate: 0.1  # image translation (+/- fraction)
# scale: 0.5  # image scale (+/- gain)
# shear: 0.0  # image shear (+/- deg)
# perspective: 0.0  # image perspective (+/- fraction), range 0-0.001
# flipud: 0.5  # image flip up-down (probability)
# fliplr: 0.5  # image flip left-right (probability)
# mosaic: 1.0  # image mosaic (probability)
# mixup: 0.0  # image mixup (probability)
# copy_paste: 0.0  # segment copy-paste (probability)

In [4]:
# train 결과 확인
path = 'D:/Dog/yolov5/runs/train/dog12/results.csv'
df = pd.read_csv(path)
df

,epoch,train/box_loss,train/obj_loss,train/cls_loss,metrics/precision,metrics/recall,metrics/mAP_0.5,metrics/mAP_0.5:0.95,val/box_loss,val/obj_loss,val/cls_loss,x/lr0,x/lr1,x/lr2
0,0,0.108170,0.030438,0.081701,0.000000,0.00000,0.000000,0.000000,0.099950,0.023176,0.076658,0.088300,0.001300,0.001300
1,1,0.094074,0.029610,0.077287,0.001049,0.19444,0.017247,0.004573,0.079230,0.022076,0.076168,0.075433,0.002433,0.002433
2,2,0.081843,0.028148,0.073499,0.002666,0.50000,0.012080,0.004304,0.071837,0.020123,0.077275,0.062288,0.003288,0.003288
3,3,0.078729,0.026634,0.071605,0.006348,0.61111,0.032620,0.009898,0.064968,0.018708,0.077026,0.048867,0.003867,0.003867
4,4,0.073708,0.026890,0.068633,0.020038,0.33333,0.043943,0.016537,0.064030,0.017530,0.076516,0.035168,0.004168,0.004168
5,5,0.068477,0.024417,0.066950,0.006280,0.69444,0.022526,0.010168,0.058859,0.016464,0.074885,0.021192,0.004191,0.004191
6,6,0.068019,0.024267,0.068923,0.008727,0.75000,0.037050,0.012387,0.063993,0.015791,0.074476,0.006938,0.003938,0.003938
7,7,0.064715,0.022515,0.068203,0.011284,0.91667,0.127930,0.042809,0.061326,0.015446,0.073747,0.003070,0.003070,0.003070
8,8,0.059321,0.023847,0.065046,0.009374,0.91667,0.105400,0.049029,0.050328,0.015684,0.071900,0.003070,0.003070,0.003070
9,9,0.052544,0.024765,0.065485,0.008812,0.91667,0.188170,0.103060,0.045527,0.015538,0.071760,0.002080,0.002080,0.002080


In [ ]:
# 학습된 모델을 이용해 test
!python D:/Dog/yolov5/detect.py --weights D:/Dog/yolov5/runs/train/dog12/weights/best.pt --source D:/Dog/noseprints/data/test/images

In [ ]:
# exp3 전부 no detection...데이터가 턱없이 모자르다는 것을 확인